In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

import dataset
assert tf.config.list_physical_devices('GPU')

Using TensorFlow backend.


In [25]:
BATCH_SIZE = 32
MAXLEN = 64

def build_model():
    EMBED_DIM = 512
    UNITS = 256

    LETTERS_SIZE = len(dataset.letters_table)
    NIQQUD_SIZE = len(dataset.niqqud_table)
    DAGESH_SIZE = len(dataset.dagesh_table)
    SIN_SIZE = len(dataset.sin_table)

    common_input = tf.keras.Input(batch_shape=(None, MAXLEN), batch_size=BATCH_SIZE)
    
    common = layers.Embedding(LETTERS_SIZE, EMBED_DIM, mask_zero=True)(common_input)
    common = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.1), merge_mode='sum')(common)
    common = layers.add([common,
                         layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.1), merge_mode='sum')(common)])

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(common)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(common)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(common))
    ]
    model = tf.keras.Model(inputs=[common_input], outputs=outputs)

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    tf.keras.utils.plot_model(model, to_file='model.png')
    model.summary()
    return model
    
model = build_model()
model.save_weights('./checkpoints/uninit')

def fit(data, learning_rates):
    x  = data.normalized_texts
    vx = data.normalized_validation
    y  = {'N': data.niqqud_texts,      'D': data.dagesh_texts,      'S': data.sin_texts,      'C': data.normalized_texts     }
    vy = {'N': data.niqqud_validation, 'D': data.dagesh_validation, 'S': data.sin_validation, 'C': data.normalized_validation}
    return model.fit(x, y, validation_data=(vx, vy), batch_size=BATCH_SIZE, epochs=len(learning_rates),
          callbacks=[
              tf.keras.callbacks.LearningRateScheduler(lambda epoch, lr: learning_rates[epoch], verbose=0),
              # tf.keras.callbacks.ModelCheckpoint(filepath='checkpoints/ckpt_{epoch}', save_weights_only=True),
          ]
    )

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 64, 512)      22528       input_9[0][0]                    
__________________________________________________________________________________________________
bidirectional_16 (Bidirectional (None, 64, 256)      1574912     embedding_8[0][0]                
__________________________________________________________________________________________________
bidirectional_17 (Bidirectional (None, 64, 256)      1050624     bidirectional_16[0][0]           
______

In [26]:
def load_data(source, maxlen=MAXLEN):
    filenames = [os.path.join('texts', f) for f in source]
    return dataset.load_file(filenames, BATCH_SIZE, 0.1, maxlen=maxlen, shuffle=True)

In [27]:
data_rabanit = load_data(['birkat_hamazon.txt', 'kuzari.txt', 'hakdama_leorot.txt', 'hartzaat_harav.txt', 'orhot_hayim.txt', 'rambam_mamre.txt', 'short_table.txt', 'tomer_dvora.txt'])

In [28]:
data_pre_modern = load_data(['elef_layla.txt', 'bialik', 'shaul_tchernichovsky', 'breslev.txt', 'itzhak_berkman', 'zevi_scharfstein', 'pesah_kaplan', 'abraham_regelson',
                             'elisha_porat', 'uriel_ofek', 'yisrael_dushman', 'zvi_zviri', 'atar_hashabat.txt', 'ali_baba.txt'])

In [69]:
data_modern = load_data(['forums', 'newspapers', 'wiki', 'blogs', 'sipurim.txt', 'ricky.txt', 'imagination.txt', 'adamtsair.txt', 'katarsis.txt'])

In [33]:
model.load_weights('./checkpoints/uninit')
history = fit(data_rabanit, [3e-3, 3e-4])
model.save_weights('./checkpoints/rabanit')

Train on 76377 samples, validate on 8487 samples
Epoch 1/2
76377/76377 [==============================] - 99s 1ms/sample - loss: 0.2589 - N_loss: 0.2000 - D_loss: 0.0526 - S_loss: 0.0062 - N_accuracy: 0.9305 - D_accuracy: 0.9802 - S_accuracy: 0.9983 - val_loss: 0.1447 - val_N_loss: 0.1093 - val_D_loss: 0.0327 - val_S_loss: 0.0027 - val_N_accuracy: 0.9625 - val_D_accuracy: 0.9882 - val_S_accuracy: 0.9994
Epoch 2/2
76377/76377 [==============================] - 100s 1ms/sample - loss: 0.1085 - N_loss: 0.0803 - D_loss: 0.0259 - S_loss: 0.0023 - N_accuracy: 0.9729 - D_accuracy: 0.9907 - S_accuracy: 0.9995 - val_loss: 0.1081 - val_N_loss: 0.0801 - val_D_loss: 0.0257 - val_S_loss: 0.0022 - val_N_accuracy: 0.9730 - val_D_accuracy: 0.9907 - val_S_accuracy: 0.9995


In [34]:
model.load_weights('./checkpoints/rabanit')
history = fit(data_pre_modern, [3e-3, 3e-4])
model.save_weights('./checkpoints/pre_modern')

Train on 84096 samples, validate on 9344 samples
Epoch 1/2
84096/84096 [==============================] - 110s 1ms/sample - loss: 0.2576 - N_loss: 0.1936 - D_loss: 0.0579 - S_loss: 0.0061 - N_accuracy: 0.9346 - D_accuracy: 0.9767 - S_accuracy: 0.9984 - val_loss: 0.2116 - val_N_loss: 0.1559 - val_D_loss: 0.0505 - val_S_loss: 0.0052 - val_N_accuracy: 0.9481 - val_D_accuracy: 0.9799 - val_S_accuracy: 0.9986
Epoch 2/2
84096/84096 [==============================] - 105s 1ms/sample - loss: 0.1743 - N_loss: 0.1268 - D_loss: 0.0431 - S_loss: 0.0044 - N_accuracy: 0.9574 - D_accuracy: 0.9828 - S_accuracy: 0.9988 - val_loss: 0.1744 - val_N_loss: 0.1263 - val_D_loss: 0.0437 - val_S_loss: 0.0044 - val_N_accuracy: 0.9580 - val_D_accuracy: 0.9829 - val_S_accuracy: 0.9989


In [70]:
model.load_weights('./checkpoints/pre_modern')
history = fit(data_modern, [3e-3, 3e-4])

Train on 9964 samples, validate on 1108 samples
Epoch 1/2
9964/9964 [==============================] - 13s 1ms/sample - loss: 0.2417 - N_loss: 0.1789 - D_loss: 0.0569 - S_loss: 0.0058 - N_accuracy: 0.9450 - D_accuracy: 0.9777 - S_accuracy: 0.9987 - val_loss: 0.1945 - val_N_loss: 0.1422 - val_D_loss: 0.0472 - val_S_loss: 0.0047 - val_N_accuracy: 0.9564 - val_D_accuracy: 0.9811 - val_S_accuracy: 0.9989
Epoch 2/2
9964/9964 [==============================] - 13s 1ms/sample - loss: 0.1449 - N_loss: 0.1044 - D_loss: 0.0365 - S_loss: 0.0039 - N_accuracy: 0.9689 - D_accuracy: 0.9859 - S_accuracy: 0.9990 - val_loss: 0.1746 - val_N_loss: 0.1268 - val_D_loss: 0.0431 - val_S_loss: 0.0043 - val_N_accuracy: 0.9616 - val_D_accuracy: 0.9826 - val_S_accuracy: 0.9990


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=2)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['D', 'N'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()

In [71]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, '.')

In [64]:
def print_predictions(data, k):
    s = slice(k*BATCH_SIZE, (k+1)*BATCH_SIZE)
    batch = data.normalized_validation[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2])]
    [expected_niqqud, expected_dagesh, expected_sin] = [data.niqqud_validation[s], data.dagesh_validation[s], data.sin_validation[s]]
    actual = data.merge(batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = data.merge(batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    for i, (a, e) in enumerate(zip(actual, expected)):
        print('מצוי: ', a)
        print('רצוי: ', e)
        print()

print_predictions(load_data(['blogs'], maxlen=MAXLEN), 0)

מצוי:  עֲשׂוּיוֹת לְמִידַּת מִדְיוּם-רַיֵיר, מֵעַל רוֹטֶב "טוֹפִי יָפְנִי", שֶׁעִיקְּרוּ חֶמְאָה
רצוי:  עֲשׂוּיוֹת לְמִידַּת מֶדְיוּם-רֵייר, מֵעַל רוֹטֶב "טוֹפִי יַפָּנִי", שֶׁעִיקָּרוֹ חֶמְאָה

מצוי:  שֶׁנִּמְצְאוּ בִּצְפוֹן הָהָר, עִם שֵׁמוֹת בְּעִבְרִית וּבֵינֵיהֶם "בֶּן יָאִיר". לֹא מִן הַנִּמְנָע
רצוי:  שֶׁנִּמְצְאוּ בִּצְפוֹן הָהָר, עִם שֵׁמוֹת בְּעִבְרִית וּבֵינֵיהֶם "בֶּן יָאִיר". לֹא מִן הַנִּמְנָע

מצוי:  חוּץ מֵהַמְּבְטָא הָרוּסִי הַמְּזוּיָּף, בְּבַקָּשָׁה, דַּי כְּבָר עִם זֶה. הַסְּצָנוֹת שֶׁבָּהֶן הִיא
רצוי:  חוּץ מֵהַמִּבְטָא הָרוּסִי הַמְּזוּיָּף, בְּבַקָּשָׁה, דַּי כְּבָר עִם זֶה. הַסְּצֵנוֹת שֶׁבָּהֶן הִיא

מצוי:  אֶת הַהוּמוֹר הָעַצְמִי עֵקֶב שִׁימּוּשׁ יֶתֶר בְּחוֹמָרִים מִן הַמּוּכָן, נִגְלָה שֶׁהַקְּלִישָׁאוֹת
רצוי:  אֶת הַהוּמוֹר הָעַצְמִי עֵקֶב שִׁימּוּשׁ יֶתֶר בְּחוֹמָרִים מִן הַמּוּכָן, נִגְלֶה שֶׁהַקְּלִישָׁאוֹת

מצוי:  נוֹסֶפֶת, הַפַּעַם שְׂמֹאלָה לְמֶרְכַּז הָעִיר, יֶשְׁנָם אֲנָשִׁים רַבִּים אֲבָל מִתּוֹךְ כּוּלָּם,
רצוי:  נוֹסֶפֶת, הַפַּעַם שְׂמֹאלָה לְמֶר